In [9]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow

import random
import json


stemmer = LancasterStemmer()
# nltk.download("punkt")

# jsonデータの読み込み
with open("intents.json") as file:
    data = json.load(file)


# 読み込んだデータの抽出
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])


# 単語のステミング
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


# bag of words
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
        
training = np.array(training)
output = np.array(output)


# モデル開発
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


# モデルのトレーニングと保存
model.fit(training, output, n_epoch=1500, batch_size=8, show_metric=True)
model.save("model.tflearn")




Training Step: 4499  | total loss: 0.94596 | time: 0.005s
| Adam | epoch: 1500 | loss: 0.94596 - acc: 0.9415 -- iter: 16/20
Training Step: 4500  | total loss: 0.85174 | time: 0.006s
| Adam | epoch: 1500 | loss: 0.85174 - acc: 0.9474 -- iter: 20/20
--
INFO:tensorflow:c:\Users\pt.yo-tatsuta\Documents\python\python3.9\ai\deeplearning\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(words.lower()) for words in s_words]
    
    for se in s_words:
        for i, q in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)

def chat():
    print("Start talking with your bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in data["intents"]:
            if tg["tag"] == tag:
                responses = tg["responses"]
                
        print(random.choice(responses))
        
chat()

Start talking with your bot (type quit to stop)!
We accept VISA, Mastercard and AMEX
We accept VISA, Mastercard and AMEX
Bye! Come back again soon.
See you later, thanks for visiting
Bye! Come back again soon.
Bye! Come back again soon.
